<br><br>

## **Import necessary Python libraries and modules**

In [1]:

# For data manipulation and analysis
import pandas as pd
import numpy as np

# For machine learning tools and evaluation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report


# For deep learning
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
import torch



To use the HuggingFace [`transformers` Python library](https://huggingface.co/transformers/installation.html), we will install it with `pip`.

In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 2.9 MB/s 
     |████████████████████████████████| 6.5 MB 32.2 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 895 kB 36.8 MB/s 
     |████████████████████████████████| 596 kB 37.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast
from transformers import Trainer, TrainingArguments

<br><br>

## **Set parameters and file paths**

In [4]:
# This is the name of the BERT model that we want to use. 
# We're using DistilBERT to save space (it's a distilled version of the full BERT model), 
# and we're going to use the cased (vs uncased) version.
model_name = 'xlm-roberta-base'  

# This is the name of the program management system for NVIDIA GPUs. We're going to send our code here.
device_name = 'cuda'       

# This is the maximum number of tokens in any document sent to BERT.
max_length = 128                                                       

# This is the name of the directory where we'll save our model. You can name it whatever you want.
#cached_model_directory_name = 'ABSA_FineTuning_BERT'  
cached_model_directory_name = 'Polarity_dev'  

In [5]:
#stiamo utlizzando la GPU?
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [6]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 448 kB 51.0 MB/s 


In [7]:
#pretokenizzazionen in morfemi delle koreane
from konlpy.tag import Okt
def pretok_ko(subset, indice):
  new_ko=[] 
  #count=0
  for index,row in subset.iterrows():
    if index < indice:
      new_ko.append(row['Sentence'])
    else: #quando arrivi all'indice di ko

      okt = Okt()
      li=okt.morphs(str(row['Sentence']))
      stringa=''
      for el in li:
        stringa+=str(el)+' '
      new_ko.append(stringa[:-1])
  subset['Sentence']=new_ko
  return subset
    


In [8]:
dev=pd.read_csv("dev_def_pol.csv")
dev=pretok_ko(dev,352)
dev

,Unnamed: 0,Sentence,Polarity_Classification
0,0,The solution given here is fantastic and a bit...,Positiva
1,1,It is well worth your time to read this one,Positiva
2,2,This book is jam-packed with many fantastical ...,Positiva
3,3,The European locations never quite come to lif...,Negativa
4,4,But the biggest problem with The Last Temptati...,Negativa
...,...,...,...
694,694,나 는 왜 이 영화 가 소통 부재 에 대한 썰로 이리 유명한지 모르겠다 . 막상 영...,Negativa
695,695,강풀 원작 영화 이웃사람 을 보고나니 문득 예전 에 봤던 이 영화 가 떠올랐다 . ...,Negativa
696,696,누를 땐 니 맘 일지 몰라도 결과 는 모른다하 경준 씨,Negativa
697,697,노잼 노잼 인터스텔라 보다 별로 라는 앤 뭐 지 ? 그냥 놀래 키 는 거 랑 잔인하...,Negativa


In [9]:
test=pd.read_csv("test_def_pol.csv")
test=pretok_ko(test,699)
test

,Unnamed: 0,Sentence,Polarity_Classification
0,0,This is one of the books that my dad read out ...,Positiva
1,1,Garth Nix's 'Mister Monday' was a highly cleve...,Positiva
2,2,"Includes affirmations, visualizations, and pra...",Positiva
3,3,Monday was fantasy,Neutra
4,4,Weird and dangerous creatures are hunting him ...,Neutra
...,...,...,...
1387,1387,"달리는 동안 그 는 이 것 을 작은 열쇠 라고 하는 것 을 갖게 되었고 , 그것 은...",Neutra
1388,1388,아벨라 부터 들어간 기아 순정 이긴 한데 프라이드 는 잘 모르 겠 네요 ㅋㅋ 끝물 ...,Neutra
1389,1389,이전 이나 변함 없다 .,Negativa
1390,1390,[ 3 점 ] 뭐 하자는거지 ?,Negativa


In [10]:
#creiamo le liste di train e test. 1.8k train 250 test  
train_texts=dev['Sentence']
train_labels=dev['Polarity_Classification']
test_texts=test['Sentence']
test_labels=test['Polarity_Classification']


<br><br>

## **Split the data into training and test sets**

In [11]:
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(699, 699, 1392, 1392)

Here's an example of a training label and review:

In [12]:
train_labels[0], train_texts[0]

('Positiva',
 'The solution given here is fantastic and a bit bizarre and there lies the appeal of this book')

<br><br>

## **Encode data for BERT**

We're going to transform our texts and labels into a format that BERT (via Huggingface and PyTorch) will understand. This is called *encoding* the data.

Here are the steps we need to follow:

1. The labels&mdash;in this case, Goodreads genres&mdash;need to be turned into integers rather than strings.

2. The texts&mdash;in this case, Goodreads reviews&mdash;need to be truncated if they're more than 512 tokens or padded if they're fewer than 512 tokens. The tokens, or words in the texts, also need to be separated into "word pieces" and matched to their embedding vectors.

3. We need to add special tokens to help BERT:

| BERT special token | Explanation |
| --------------| ---------|
| [CLS] | Start token of every document. |
| [SEP] | Separator between each sentence |
| [PAD] | Padding at the end of the document as many times as necessary, up to 512 tokens |
|  &#35;&#35; | Start of a "word piece" |




In [13]:
tokenizer = XLMRobertaTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [14]:
#unique_labels = set(label for label in train_labels)
#label2id = {label: id for id, label in enumerate(unique_labels)}
#id2label = {id: label for label, id in label2id.items()}

In [15]:
unique_labels={'Neutra', 'Positiva', 'Negativa'}
label2id={'Negativa': 2, 'Neutra': 0, 'Positiva': 1}
id2label={2:'Negativa', 0:'Neutra', 1:'Positiva'}

In [16]:
unique_labels

{'Negativa', 'Neutra', 'Positiva'}

In [17]:
label2id.keys()

dict_keys(['Negativa', 'Neutra', 'Positiva'])

In [18]:
id2label.keys()

dict_keys([2, 0, 1])

Now let's encode our texts and labels!

In [19]:
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings  = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)

train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]

In [20]:
type(test_labels_encoded[100])

int

In [21]:
' '.join(train_encodings[29].tokens[0:100])

'<s> ▁Arthur ▁Pen hali gon ▁and ▁Su zy ▁Blue ▁are ▁two ▁amazing ▁characters , ▁Arthur : ▁a ▁normal ▁boy , ▁as th matic , ▁not ▁too ▁confident ▁about ▁himself , ▁intelligent ▁and ▁thought ful </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [22]:
' '.join(test_encodings[0].tokens[0:100])

'<s> ▁This ▁is ▁one ▁of ▁the ▁books ▁that ▁my ▁dad ▁read ▁out ▁lo ud ▁to ▁me ▁at ▁bed time , ▁but ▁whenever ▁he ▁had ▁to ▁put ▁it ▁down , ▁I ▁begge d ▁him ▁to ▁read ▁more </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

**Examine the training labels after encoding**

In [23]:
set(train_labels_encoded)

{0, 1, 2}

**Examine the test labels after encoding**

In [24]:
set(test_labels_encoded)

{0, 1, 2}

<br><br>

## **Make a custom Torch dataset**

Here we combine the encoded labels and texts into dataset objects. We use the custom Torch `MyDataSet` class to make a `train_dataset` object from  the `train_encodings` and `train_labels_encoded`. We also make a `test_dataset` object from `test_encodings`, and `test_labels_encoded`.

In [25]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [26]:
train_dataset = MyDataset(train_encodings, train_labels_encoded)
test_dataset = MyDataset(test_encodings, test_labels_encoded)

In [27]:
' '.join(train_dataset.encodings[0].tokens[0:100])

'<s> ▁The ▁solution ▁given ▁here ▁is ▁fantastic ▁and ▁a ▁bit ▁bizarre ▁and ▁there ▁li es ▁the ▁appeal ▁of ▁this ▁book </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [28]:
' '.join(test_dataset.encodings[1].tokens[0:100])

"<s> ▁Gar th ▁Ni x ' s ▁' M ister ▁Monday ' ▁was ▁a ▁highly ▁c lever , ▁creative , ▁and ▁entertain ing ▁read ▁that ▁had ▁me ▁up ▁into ▁the ▁we e ▁hours ▁of ▁the ▁morning ▁following ▁Arthur ' s ▁adventure s ▁in ▁the ▁House </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>"

<br><br>

## **Load pre-trained BERT model**

In [29]:
# The model_name needs to match the name used for the tokenizer above.
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

<br><br>

## **Set the BERT fine-tuning parameters**

These are the arguments we'll set in the HuggingFace TrainingArguments objects, which we'll then pass to the HuggingFace Trainer object. There are many more possible arguments, but here we highlight the basics and some common gotchas.

When training your own model, you should search over these parameters to find the best settings for your particular dataset. You should use a held-out set of validation data for this step.

| Parameter | Explanation |
|-----------| ------------|
| num_train_epochs | total number of training epochs (how many times to pass through the entire dataset; too much can cause overfitting) |
| per_device_train_batch_size | batch size per device during training |
| per_device_eval_batch_size |  batch size for evaluation |
|  warmup_steps |  number of warmup steps for learning rate scheduler (set lower because of small dataset size) |
| weight_decay | strength of weight decay (reduces size of weights, like regularization) |
| output_dir | output directory for the fine-tuned model and configuration files |
| logging_dir | directory for storing logs |
| logging_steps | how often to print logging output (so that we can stop training early if the loss isn't going down) |
| evaluation_strategy | evaluate while training so that we can see the accuracy going up |

In [30]:
training_args = [TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    learning_rate=5e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'
    # evaluate during fine-tuning so that we can see progress
), TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
), 
TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    learning_rate=3e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
),
TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=5e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
), 
TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
),
TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=3e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
)]

<br><br>

## **Fine-tune the BERT model**

First, we define a custom evaluation function that returns the accuracy. You could modify this function to return precision, recall, F1, and/or other metrics.

In [31]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

Then we create a HuggingFace `Trainer` object using the `TrainingArguments` object that we created above. We also send our `compute_metrics` function to the `Trainer` object, along with our test and train datasets.

**Note:** This is what we've been aiming for this whole time! All the work of tokenizing, creating datasets, and setting the training arguments was for this cell.

In [32]:
for argomenti_train in training_args:
  trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=argomenti_train,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset (usually a validation set; here we just send our test set)
    compute_metrics=compute_metrics      # our custom evaluation function 
    )
  
  trainer.train()

  trainer.evaluate()

  predicted_results = trainer.predict(test_dataset)

  predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
  predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
  predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

  print(classification_report(test_labels, 
                            predicted_labels))
  print('\n')


#meglio prediligere learn rate bassi ad ogni modo
#https://arxiv.org/pdf/1905.05583.pdf

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 699
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 88


Step,Training Loss,Validation Loss


Saving model checkpoint to ./results/checkpoint-22
Configuration saved in ./results/checkpoint-22/config.json
Model weights saved in ./results/checkpoint-22/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-44
Configuration saved in ./results/checkpoint-44/config.json
Model weights saved in ./results/checkpoint-44/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-66
Configuration saved in ./results/checkpoint-66/config.json
Model weights saved in ./results/checkpoint-66/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-88
Configuration saved in ./results/checkpoint-88/config.json
Model weights saved in ./results/checkpoint-88/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 1392
  Batch size = 32


***** Running Prediction *****
  Num examples = 1392
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 699
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 88


              precision    recall  f1-score   support

    Negativa       0.66      0.35      0.46       460
      Neutra       0.62      0.80      0.70       467
    Positiva       0.54      0.63      0.58       465

    accuracy                           0.59      1392
   macro avg       0.61      0.59      0.58      1392
weighted avg       0.61      0.59      0.58      1392





...

              precision    recall  f1-score   support

    Negativa       0.77      0.57      0.65       460
      Neutra       0.74      0.74      0.74       467
    Positiva       0.65      0.82      0.73       465

    accuracy                           0.71      1392
   macro avg       0.72      0.71      0.71      1392
weighted avg       0.72      0.71      0.71      1392





Step,Training Loss,Validation Loss


Saving model checkpoint to ./results/checkpoint-44
Configuration saved in ./results/checkpoint-44/config.json
Model weights saved in ./results/checkpoint-44/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-88
Configuration saved in ./results/checkpoint-88/config.json
Model weights saved in ./results/checkpoint-88/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-132
Configuration saved in ./results/checkpoint-132/config.json
Model weights saved in ./results/checkpoint-132/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-176
Configuration saved in ./results/checkpoint-176/config.json
Model weights saved in ./results/checkpoint-176/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 1392
  Batch size = 16


***** Running Prediction *****
  Num examples = 1392
  Batch size = 16


              precision    recall  f1-score   support

    Negativa       0.76      0.56      0.65       460
      Neutra       0.77      0.69      0.73       467
    Positiva       0.61      0.83      0.71       465

    accuracy                           0.70      1392
   macro avg       0.71      0.70      0.69      1392
weighted avg       0.71      0.70      0.69      1392





In [33]:
#qui il compound 5 è il migliore